In [1]:
import os

import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE


In [2]:
abt = pd.read_csv("../Resources/analytical_base_table.csv")
abt.head()

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,France,Female,42,2,0.00,1,1,101348.88,1
1,Spain,Female,41,1,83807.86,1,1,112542.58,0
2,France,Female,42,8,159660.80,3,0,113931.57,1
3,France,Female,39,1,0.00,2,0,93826.63,0
4,Spain,Female,43,2,125510.82,1,1,79084.10,0


## Models Training

## Let's start by splitting our dataframe into separate objects:

y for the target varibale

X for the input features

In [3]:
#Separate dataframe into separate object

# Object for target variable
y = abt.Exited

# object for input features
X = abt.drop(['Exited'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)


(10000, 8) (10000,)


In [4]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'IsActiveMember',
 'EstimatedSalary']

In [5]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Geography', 'Gender']

In [6]:
def class_count(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    abt2 = pd.DataFrame(np.array(kv).T, columns=['Exited','Count'])
    abt2['Count'] = abt2['Count'].astype('int64')
    abt2['%'] = round(abt2['Count'] / a.shape[0] * 100, 2)
    return abt2.sort_values('Count',ascending=False)

In [7]:
class_count(y)


,Exited,Count,%
1,0,7963,79.63
0,1,2037,20.37


## Create a Train Test Split

We will continue with splitting our data into separate training and test sets.

30% of observations will be set aside for the test set

the rest, 70%, will be used as the training set

In [8]:
random_state = 8

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=abt.Exited)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 6347 to 2994
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geography        7000 non-null   object 
 1   Gender           7000 non-null   object 
 2   Age              7000 non-null   int64  
 3   Tenure           7000 non-null   int64  
 4   Balance          7000 non-null   float64
 5   NumOfProducts    7000 non-null   int64  
 6   IsActiveMember   7000 non-null   int64  
 7   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(4), object(2)
memory usage: 492.2+ KB


## Pre-processing Pipeline
 
## Scale numerical data and encode categorical data
Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder Create lists of indexes from the list of column namesNeed to be numeric not string to specify columns name in column transformer

In [10]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)  

[2, 3, 4, 5, 6, 7]


In [11]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features) 

[0, 1]


In [12]:
# Define column transformer
# Need to be numeric not string to specify columns name 
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(), cat_features)
)
preprocess

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [2, 3, 4, 5, 6, 7]),
                                ('onehotencoder', OneHotEncoder(), [0, 1])])

In [13]:
# Import classifier
from sklearn import tree

# Define model with pipeline
model = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  tree.DecisionTreeClassifier(random_state=random_state))

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [2, 3, 4, 5, 6, 7]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(), [0, 1])])),
                ('smote', SMOTE(random_state=8)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=8))])

In [14]:
 # Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
dt_param_grid = {
            'decisiontreeclassifier__max_leaf_nodes': [2, 10, 20, 30], 
            'decisiontreeclassifier__min_samples_split': [2, 3, 4],
            'decisiontreeclassifier__criterion': ['gini', 'entropy']
            }

dt_grid = GridSearchCV(model, dt_param_grid, verbose=3, cv= 5, scoring='accuracy')

In [15]:
X_train = X_train.values
X_test = X_test.values

In [16]:
dt_grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=2;, score=0.693 total time=   0.0s
[CV 2/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=2;, score=0.714 total time=   0.0s
[CV 3/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=2;, score=0.702 total time=   0.0s
[CV 4/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=2;, score=0.701 total time=   0.0s
[CV 5/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=2;, score=0.694 total time=   0.0s


[CV 1/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=3;, score=0.693 total time=   0.0s
[CV 2/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=3;, score=0.714 total time=   0.0s
[CV 3/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=3;, score=0.702 total time=   0.0s
[CV 4/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=3;, score=0.701 total time=   0.0s
[CV 5/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=3;, score=0.694 total time=   0.0s
[CV 1/5] END decisiontreeclassifier__criterion=gini, decisiontreeclassifier__max_leaf_nodes=2, decisiontreeclassifier__min_samples_split=4;,

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [2, 3,
                                                                          4, 5,
                                                                          6,
                                                                          7]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         [0,
                                                                          1])])),
                                       ('smote', SMOTE(random_state=8)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier(random_state=8))]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_leaf_nodes': [2, 10, 20,
                                                                    30],
                         'decisiontreeclassifier__min_samples_split': [2, 3,
                                                                       4]},
             scoring='accuracy', verbose=3)

In [17]:
print(dt_grid.best_params_)

{'decisiontreeclassifier__criterion': 'entropy', 'decisiontreeclassifier__max_leaf_nodes': 30, 'decisiontreeclassifier__min_samples_split': 2}


In [18]:
print(dt_grid.best_score_)

0.7984285714285713


In [19]:
print(f"Training Data Score: {dt_grid.score(X_train, y_train)}")
print(f"Testing Data Score: {dt_grid.score(X_test, y_test)}")

Training Data Score: 0.8192857142857143
Testing Data Score: 0.8206666666666667


In [20]:
predictions = dt_grid.predict(X_test)
print(f"First 8 Predictions:   {predictions[:8]}")
print(f"First 8 Actual labels: {y_test[:8].tolist()}")

First 8 Predictions:   [1 0 0 1 0 0 0 0]
First 8 Actual labels: [1, 0, 0, 1, 1, 0, 0, 0]


In [21]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1,1
1,0,0
2,0,0
3,1,1
4,0,1
...,...,...
2995,0,0
2996,0,0
2997,0,0
2998,0,0


In [22]:
cm = confusion_matrix(y_test, predictions)
print(cm)

[[2069  320]
 [ 218  393]]


In [23]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.87 0.13]
 [0.36 0.64]]


In [24]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      2389
           1       0.55      0.64      0.59       611

    accuracy                           0.82      3000
   macro avg       0.73      0.75      0.74      3000
weighted avg       0.83      0.82      0.83      3000



In [25]:
pred = dt_grid.predict(X_test[:1])

In [26]:
print(f"Predicted classes: {pred}")
print(f"Actual Labels: {list(y_test[:1])}")

Predicted classes: [1]
Actual Labels: [1]


In [27]:
import joblib

filename = '../models/nate_decision_tree.sav'
joblib.dump(dt_grid, filename)

['../models/nate_decision_tree.sav']

In [28]:
dt_model = joblib.load(filename)
print(dt_model.score(X_test, y_test))

0.8206666666666667
